# Mr. TyDi

## 概要

**データセットを以下の2種類のデータとして再構築する**  

`df_qa`: Queryとその回答に必要な参考情報のID
- Query: 質問
- ID: 回答に必要な参考情報のID

`df_ref`:
- ID: 回答に必要な参考情報のID
Reference: 参考情報

## imports

In [41]:
import os
import copy
import pandas as pd
from typing import List

import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option("display.width", 100)
pd.set_option("display.max_colwidth", None)

## 1. Hugging Faceからコーパスををダウンロード
- https://github.com/castorini/mr.tydi

In [42]:
# !pip install datasets

In [43]:
from datasets import load_dataset
corpus_dataset = load_dataset(
    "castorini/mr-tydi-corpus",
    "japanese",
    split="train",
    trust_remote_code=True,
)

Using the latest cached version of the module from /Users/ryoyakatafuchi/.cache/huggingface/modules/datasets_modules/datasets/castorini--mr-tydi-corpus/e03356656eff551e40ab09e74d102c56320e0fbf44f0219f1dd20d524663663c (last modified on Thu Jul 25 01:09:58 2024) since it couldn't be found locally at castorini/mr-tydi-corpus, or remotely on the Hugging Face Hub.


In [58]:
df_ref = corpus_dataset.to_pandas()

In [65]:
df_ref = df_ref.rename(columns={"docid":"ID","text":"Reference"})

In [66]:
df_ref.head(2)

,ID,title,Reference
0,5#0,アンパサンド,"アンパサンド (&、英語名：) とは並立助詞「…と…」を意味する記号である。ラテン語の の合字で、Trebuchet MSフォントでは、と表示され ""et"" の合字であることが容易にわかる。ampersa、すなわち ""and per se and""、その意味は""and [the symbol which] by itself [is] and""である。"
1,5#1,アンパサンド,その使用は1世紀に遡ることができ、5世紀中葉から現代に至るまでの変遷がわかる。\nZ に続くラテン文字アルファベットの27字目とされた時期もある。


## 2. Hugging FaceからQAデータセットをダウンロード

In [46]:
qa_dataset = load_dataset(
    "castorini/mr-tydi",
    "japanese",
    split="test",
    trust_remote_code=True,
)

In [47]:
df_qa = qa_dataset.to_pandas()

## 3. docid の抽出

In [48]:
def get_docid(data):
    return [d["docid"] for d in data]
        
df_qa["ID"] = df_qa["positive_passages"].map(get_docid)

In [49]:
df_qa["ID"].head()

0               [88104#2]
1             [1635294#0]
2               [2279#12]
3    [477816#0, 477816#3]
4              [33474#12]
Name: ID, dtype: object

In [61]:
# df_qa内のdocidが全てdf_ref内のdoc_idに含まれているかを確認
docids = df_ref["ID"].unique()
docids_qa = sum(df_qa["ID"].values,[])
set(docids_qa) - set(docids)

set()

## 4. 保存

In [63]:
df_ref.head()

,ID,title,text
0,5#0,アンパサンド,"アンパサンド (&、英語名：) とは並立助詞「…と…」を意味する記号である。ラテン語の の合字で、Trebuchet MSフォントでは、と表示され ""et"" の合字であることが容易にわかる。ampersa、すなわち ""and per se and""、その意味は""and [the symbol which] by itself [is] and""である。"
1,5#1,アンパサンド,その使用は1世紀に遡ることができ、5世紀中葉から現代に至るまでの変遷がわかる。\nZ に続くラテン文字アルファベットの27字目とされた時期もある。
2,5#2,アンパサンド,"アンパサンドと同じ役割を果たす文字に「のet」と呼ばれる、数字の「7」に似た記号があった(, U+204A)。この記号は現在もゲール文字で使われている。"
3,5#3,アンパサンド,"記号名の「アンパサンド」は、ラテン語まじりの英語「& はそれ自身 ""and"" を表す」(& per se and) のくずれた形である。英語以外の言語での名称は多様である。"
4,5#4,アンパサンド,日常的な手書きの場合、欧米でアンパサンドは「ε」に縦線を引く単純化されたものが使われることがある。


In [67]:
dataset_dir = "./dataset/05_mr-tydi"
# os.makedirs(dataset_dir)
df_ref.to_csv("./dataset/05_mr-tydi/Reference.csv")
# df_qa.to_csv("./dataset/05_mr-tydi/QA.csv")